# Submission Information:

### Team Member 1:
* UNI: mf3090
* Name: Michael Figueroa

### Team Member 2 [optional]:
* UNI: baa2146
* Name: Brett Averso

# Step0 - Import Libraries, Load Data [0 points]

This is the basic step where you can load the data and create train and test sets for internal validation as per your convinience.

In [1]:
cd data

/Users/brettaverso/Desktop/homework-iii-APM-master/homework-iii-APM-master/data


In [2]:
#write code below, you can make multiple cells
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Imputer, OneHotEncoder, StandardScaler
from pandas.tools.plotting import scatter_matrix

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

In [33]:
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# CHECK P 295 FOR THIS CURVE TUTORIAL
from sklearn.metrics import roc_curve 

# Step1 - Exploration and Preparation [10 points]

In this step, we expect you to look into the data and try to understand it before modeling. This understanding may lead to some basic data preparation steps which are common across the two model sets required.

In [5]:
#write code below, you can make multiple cells

# NOTE: WE CAN SET NA VALUES HERE TO LOOK AT WHICH ROWS HAVE NA
train = pd.read_csv('data.csv', sep=',', na_values=[999, 9999, 99999, 999999, 9999999])
test = pd.read_csv('holdout.csv', sep=',')

In [86]:
def preprocess(train, test):
    #dropping features: test id, duration, prev_days, subscriptions
    #drop duration column (we do not have this information before a new call. we drop to avoid an overly optimistic model)
    #dropping prev_days because the feature contains 31707 null values (80% missing). Not likely an informative feature.
    test_ID = test.ID
    y_train = train.subscribed
    test = test.drop(['ID','duration','prev_days'], axis = 1)
    train = train.drop(['duration','prev_days','subscribed'], axis = 1)
    
    #assign binary for no and yes
    y_train.replace('no',0, inplace=True)
    y_train.replace('yes',1, inplace=True)
    
    #creating dummy variables for categorical features
    dummy_train = pd.get_dummies(train,dummy_na=False)
    dummy_test = pd.get_dummies(test,dummy_na=False)
    
    #noted test set is missing a feature, imputing a 0 vector for this feature at the appropriate column index.
    list(dummy_test)[33]
    dummy_test.insert(34, 'credit_default_yes', 0)
    
    return dummy_train, dummy_test, y_train, test_ID

In [90]:
dummy_train, dummy_test, y_train, test_ID = preprocess(train, test)

In [89]:
def scale_continuous(dummy_train, dummy_test):

    # scaling the continuous data
    continuous_train = dummy_train.select_dtypes(include=['float'])
    categorical_train = dummy_train.select_dtypes(exclude=['float'])

    continuous_test = dummy_test.select_dtypes(include=['float'])
    categorical_test = dummy_test.select_dtypes(exclude=['float'])

    # note scalar fits to train only. transforms train and test.
    scaler = StandardScaler()
    dummy_train_scaled = scaler.fit_transform(continuous_train)
    dummy_test_scaled = scaler.transform(continuous_test)
    dummy_train_scaled = pd.DataFrame(dummy_train_scaled,columns=list(continuous_train))
    dummy_test_scaled = pd.DataFrame(dummy_test_scaled,columns=list(continuous_test))

    # Combining continuous and dummy features back into a single train data frame.
    X_train = pd.concat([dummy_train_scaled, categorical_train],axis=1)
    X_test = pd.concat([dummy_test_scaled,categorical_test],axis=1)
    
    return X_train, X_test

In [25]:
X_train, X_test = scale_continuous(dummy_train, dummy_test)

In [27]:
'''# 999 ENCODES MISSING (CHECK FOR OTHERS)
for i in list(test):
    print(test[i].value_counts())

#verifying the existence of any null values other than 'unknown'
dummy_train.isnull().any()'''

"# 999 ENCODES MISSING (CHECK FOR OTHERS)\nfor i in list(test):\n    print(test[i].value_counts())\n\n#verifying the existence of any null values other than 'unknown'\ndummy_train.isnull().any()"

In [ ]:
'''# we have an imbalanced dataset (~ 10 times more yes classes)
print(" -----%-------")
print(train['subscribed'].value_counts())'''

In [ ]:
'''# show histograms of the data for final submission
train.hist(bins=30)'''

In [ ]:
'''train.describe()'''

In [10]:
'''#reviewing the listing of categorical data prior to executing get_dummies
categoricals = list(train.select_dtypes(include=['object']))
categoricals'''

['job',
 'marital_status',
 'education',
 'credit_default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'prev_outcomes',
 'subscribed']

# Step2 - ModelSet1 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set1:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. Any classification algorithm covered in class apart from tree-based models can be tested here.

In [81]:
def kaggle_output(y_prob):
    y_prob = y_prob.ix[:,[1]]
    y_prob = pd.concat([test_ID, y_prob], axis = 1)
    y_prob.columns = ['ID', 'subscribed']
    y_prob.to_csv('output.csv', index=False)

In [74]:
def model_1(X_train, y_train, X_test):
    logit = LogisticRegression()
    pipe = make_pipeline(logit)
    param_grid = {'logisticregression__C': [.01, .1, 1.0, 10, 100]}
    grid = GridSearchCV(pipe, param_grid, cv = 2)
    grid.fit(X_train, y_train)
    print(grid.score(X_train, y_train))
    scores = cross_val_score(grid, X_train, y_train, cv=5, scoring = 'roc_auc')
    print(np.mean(scores))
    print(grid.best_estimator_.named_steps['logisticregression'])
    y_prob = grid.best_estimator_.named_steps['logisticregression'].predict_proba(X_test)
    y_prob = pd.DataFrame(y_prob)
    return y_prob
    

In [79]:
y_prob = model_1(X_train, y_train, X_test)

0.899908952959
0.790927439798
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [82]:
ls

data.csv*              holdout.csv            sample_submission.csv
data_dictionary.txt*   logistic.csv


In [91]:
kaggle_output(y_prob)

In [ ]:
GridSearchCV.best_estimator

In [ ]:
def model_2(X_train, y_train, X_test):
    

# Step3 - ModelSet2 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set2:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. We encourage you to try decition tree, random forest and gradient boosted tree methods here and pick the one which you think works best.

In [ ]:
#write code below, you can make multiple cells

# Step4 - Ensemble [20 points + 10 Bonus points]

In this step, we expect you to use the models created before and create new predictions. You should definitely try poor man's stacking but we encourage you to think of different ensemble techniques as well. We will judge your creativity and improvement in model performance using ensemble models and you can potentially earn 10 bonus points here.

In [ ]:
#write code below, you can make multiple cells
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
X = X_train
y = y_train
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X, y)
predictions1 = (eclf1.predict(X))
eclf2 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
eclf2 = eclf2.fit(X, y)
predictions2 = (eclf2.predict(X))
eclf3 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft', weights=[2,1,1])
eclf3 = eclf3.fit(X, y)
predictions3 = (eclf3.predict(X))

In [ ]:
eclf1.score(X,y)